In [26]:
import torch
from ultralytics import YOLOv10
# Пример использования CUDA или 
import os
import cv2
import re
import pandas as pd

In [8]:
model = YOLOv10('/home/andrew/works/shpad_add/Detection-of-welding-seams/train_yolo/runs/detect/yolov10x_custom13/weights/best.pt')
 

In [39]:
folder_path = '/home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images'

class_mapping = {
    0: "adj",
    1: "int",
    2: "geo",
    3: "pro",
    4: "non"
}

results_list = []

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# Получение списка всех изображений в папке и сортировка
image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))], key=natural_sort_key)

print("Files to process:")
print(image_files)

# Список для хранения результатов
results_list = []

# Обход всех изображений в папке и выполнение предсказания
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    
    # Загрузка изображения для получения размеров
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    # Выполнение предсказания
    results = model.predict(image_path)
    
    # Получение боксов
    boxes = results[0].boxes.xyxy  # Координаты боксов в формате (xmin, ymin, xmax, ymax)
    scores = results[0].boxes.conf  # Уверенности предсказаний
    class_ids = results[0].boxes.cls  # Классы предсказаний
    
    # Отладочный вывод
    print(f"Processing file: {image_file}")
    print(f"Detected {len(boxes)} objects")

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i].cpu().numpy()  # Преобразование в numpy массив и извлечение координат
        class_id = int(class_ids[i].cpu().numpy())  # Класс
        confidence = scores[i].cpu().numpy()  # Уверенность
        
        # Получение текстового имени класса
        class_name = class_mapping[class_id]
        
        # Вычисление относительных координат и размеров
        rel_x = (x1 + x2) / 2 / width
        rel_y = (y1 + y2) / 2 / height
        rel_width = (x2 - x1) / width
        rel_height = (y2 - y1) / height
        
        # Добавление данных в список результатов
        results_list.append({
            'filename': image_file,
            'class_id': class_name,
            'rel_x': rel_x,
            'rel_y': rel_y,
            'width': rel_width,
            'height': rel_height
        })


image 1/1 /home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (1).jpg: 384x640 7 adjs, 29.9ms
Speed: 19.4ms preprocess, 29.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (2).jpg: 384x640 5 adjs, 15.2ms
Speed: 2.1ms preprocess, 15.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (3).jpg: 384x640 5 adjs, 16.7ms
Speed: 2.2ms preprocess, 16.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (4).jpg: 384x640 7 adjs, 16.9ms
Speed: 2.2ms preprocess, 16.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (5).jpg: 384x640 7 adjs, 17.

In [40]:
df = pd.DataFrame(results_list)
df

,filename,class_id,rel_x,rel_y,width,height
0,1 (1),"[tensor(0., device='cuda:0'), tensor(0., devic...",0.388977,0.145927,0.031684,0.075023
1,1 (1),"[tensor(0., device='cuda:0'), tensor(0., devic...",0.799644,0.315085,0.028959,0.101678
2,1 (1),"[tensor(0., device='cuda:0'), tensor(0., devic...",0.422728,0.316598,0.020165,0.049684
3,1 (1),"[tensor(0., device='cuda:0'), tensor(0., devic...",0.384856,0.478960,0.021821,0.039924
4,1 (1),"[tensor(0., device='cuda:0'), tensor(0., devic...",0.384297,0.055474,0.028944,0.033257
...,...,...,...,...,...,...
3389,12 (78),"[tensor(1., device='cuda:0')]",0.643544,0.505200,0.108740,0.072467
3390,12 (79),"[tensor(1., device='cuda:0')]",0.639804,0.608071,0.105641,0.072454
3391,12 (80),"[tensor(1., device='cuda:0')]",0.637711,0.724083,0.107818,0.081163
3392,12 (81),"[tensor(1., device='cuda:0')]",0.630053,0.825423,0.096898,0.081327


In [32]:
df['filename'].nunique()

988

In [36]:
df = pd.DataFrame(results_list)
csv_save_path = 'submission.csv'
df.to_csv(csv_save_path, index=False, sep=';')

print(f'Results saved to {csv_save_path}')

Results saved to submission.csv


In [11]:
results = model.predict('/home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (1).jpg')


image 1/1 /home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images/1 (1).jpg: 384x640 7 adjs, 26.9ms
Speed: 19.2ms preprocess, 26.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


In [13]:
results[0].boxes.xyxy

tensor([[ 716.4193,  117.0887,  777.2529,  198.1134],
        [1507.5168,  285.3860, 1563.1176,  395.1985],
        [ 792.2783,  315.0966,  830.9954,  368.7551],
        [ 717.9758,  495.7180,  759.8721,  538.8358],
        [ 710.0645,   41.9534,  765.6374,   77.8712],
        [ 771.9554, 1030.5179,  812.0909, 1065.2615],
        [ 786.1551,  311.2524,  834.0049,  371.5467]], device='cuda:0')